In [26]:
import datetime as dt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

In [27]:
data = pd.read_csv('data/preprocessed_data.csv')

In [28]:
location_le = preprocessing.LabelEncoder()
crime_le = preprocessing.LabelEncoder()
neighborhood_le = preprocessing.LabelEncoder()
day_of_week_le = preprocessing.LabelEncoder()

location_le.fit(data.location.unique().tolist())
crime_le.fit(data.crime.unique().tolist())
neighborhood_le.fit(data.neighborhood.unique().tolist())
day_of_week_le.fit(data.day_of_week.unique().tolist())

data.location = location_le.transform(data.location.tolist())
data.crime = crime_le.transform(data.crime.tolist())
data.neighborhood = neighborhood_le.transform(data.neighborhood.tolist())
data.day_of_week = day_of_week_le.transform(data.day_of_week.tolist())

In [29]:
X = data.drop(['crime'], axis=1)
y = data['crime']


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 4)

In [31]:
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

n_hidden_1 = 16
n_hidden_2 = 16
num_input = 9
num_classes = 11

tf_X = tf.placeholder("float", [None, num_input])
tf_y = tf.placeholder("float", [None, num_classes])


weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def neural_net(x):
    layer_1 = tf.add(tf.matmul(x,weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


logit = neural_net(tf_X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logit, labels=tf_y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(logit, 1), tf.argmax(tf_y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [32]:
def ceil(a, b):
    return -(-a//b)

In [44]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_steps+1):
        n_sample = len(X_train)
        top_sample = ceil(n_sample, ceil(n_sample, batch_size))
        
        for i in range(ceil(n_sample, top_sample)):
            mini_x = X_train[i * top_sample: (i+1) * top_sample]
            mini_y = y_train[i * top_sample: (i+1) * top_sample]
            sess.run(train_op, feed_dixt={X: mini_x, Y: mini_y})
            if step % display_step == 0 or step == 1:
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X:mini_x, Y: mini_y})
                print("Step " + str(step) + ", minibatch loss = " + \
                      "{:.4f}".format(loss) + ", training accuracy = " + \
                      "{:.3f}".format(acc))
                
    print("Optimization finished")
    print("testing accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

NameError: name 'Y' is not defined